### IMPORTS:

In [208]:
# importações iniciais padrão FIFA 2025
import pandas as pd
import numpy as np
import os
from datetime import datetime


### READING DATA:


In [209]:
DATA_DIR = "./data"

def read_data(data_dir):

    payers = pd.read_csv(f"{data_dir}/payers-v1.csv")
    terminals = pd.read_csv(f"{data_dir}/seller_terminals-v1.csv")
    transactions = pd.read_csv(f"{data_dir}/transactions_train-v1.csv")
    return payers, terminals, transactions

payers, terminals, transactions = read_data(DATA_DIR)

In [210]:
payers.head()

,card_hash,card_bin,card_first_transaction
0,6579071715002803,355137,2016-03-20
1,4814157718073,546566,2016-04-09
2,4717474907925,436231,2016-04-17
3,4747611558497388,358161,2016-05-08
4,4689529748023353,655698,2016-06-03


In [211]:
terminals.head()

,terminal_id,latitude,longitude,terminal_operation_start,terminal_soft_descriptor
0,0,-23.601328,-46.707536,2016-05-06,Urban Fiber 132
1,1,-23.505868,-46.668860,2016-06-10,Express Rentals 22
2,2,-23.554972,-46.618841,2016-06-12,Dream Home Decor 4
3,3,-15.851810,-47.946997,2016-06-13,Urban Spa 108
4,4,-2.996113,-59.963312,2016-06-13,Sportswear Hub 93


In [212]:
transactions.head()

,transaction_id,tx_datetime,tx_date,tx_time,tx_amount,card_id,terminal_id,is_transactional_fraud,is_fraud,tx_fraud_report_date
0,1,2018-01-01 00:00:10,2018-01-01,00:00:10,117.12,4983922282101,1113,0,False,NaN
1,2,2018-01-01 00:00:38,2018-01-01,00:00:38,57.70,38435765939628,790,0,False,NaN
2,3,2018-01-01 00:00:38,2018-01-01,00:00:38,41.50,4053973451359964262,157,0,False,NaN
3,4,2018-01-01 00:00:41,2018-01-01,00:00:41,47.72,370458387500772,850,0,False,NaN
4,5,2018-01-01 00:01:46,2018-01-01,00:01:46,14.53,180022431672084,32,0,False,NaN


### AGRUPA DATAFRAMES

In [213]:
def concat_dataframes(payers, terminals, transactions):

    df = transactions.merge(payers, left_on="card_id", right_on="card_hash", how="left")
    df = df.merge(terminals, on="terminal_id", how="left")
    return df

df = concat_dataframes(payers, terminals, transactions)
df = df.sample(n=500000, random_state=42)
print(f"{df.shape}")

df.head()

(500000, 17)


,transaction_id,tx_datetime,tx_date,tx_time,tx_amount,card_id,terminal_id,is_transactional_fraud,is_fraud,tx_fraud_report_date,card_hash,card_bin,card_first_transaction,latitude,longitude,terminal_operation_start,terminal_soft_descriptor
2027872,2027873,2018-03-17 17:26:01,2018-03-17,17:26:01,44.98,3512837025620287,253,0,False,NaN,3512837025620287,546566,2018-03-09,-15.716701,-47.855818,2017-03-03,Active Life 81
1851620,1851621,2018-03-11 19:42:45,2018-03-11,19:42:45,85.28,30092889305992,254,0,False,NaN,30092889305992,234005,2018-01-16,-15.724543,-47.876550,2017-03-03,Elegant Carpets 94
55485,55486,2018-01-03 10:17:41,2018-01-03,10:17:41,88.96,4867771109093320764,57,0,False,NaN,4867771109093320764,241274,2017-06-30,-15.779615,-47.891527,2016-10-14,Organic Beauty 101
3518696,3518697,2018-05-03 14:47:05,2018-05-03,14:47:05,167.73,5277997722226463,556,0,False,NaN,5277997722226463,676257,2017-05-10,-15.823158,-47.846085,2017-06-07,Elite Hair Salon 126
3228231,3228232,2018-04-24 19:22:43,2018-04-24,19:22:43,42.64,3558870140738788,1818,0,False,NaN,3558870140738788,476320,2017-08-24,-3.047734,-60.020102,2018-03-15,Heritage Walks 74


### CONVERTENDO DATAS

In [214]:
def convert_dates(df):
    
    df["tx_datetime"] = pd.to_datetime(df["tx_datetime"])
    df["tx_time"] = pd.to_datetime(df["tx_time"]).dt.time
    df["tx_date"] = pd.to_datetime(df["tx_date"]).dt.date
    df["card_first_transaction"] = pd.to_datetime(df["card_first_transaction"]).dt.date
    df["terminal_operation_start"] = pd.to_datetime(df["terminal_operation_start"]).dt.date

convert_dates(df)

df.head()

C:\Users\luigi\AppData\Local\Temp\ipykernel_30324\3871766388.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["tx_time"] = pd.to_datetime(df["tx_time"]).dt.time


,transaction_id,tx_datetime,tx_date,tx_time,tx_amount,card_id,terminal_id,is_transactional_fraud,is_fraud,tx_fraud_report_date,card_hash,card_bin,card_first_transaction,latitude,longitude,terminal_operation_start,terminal_soft_descriptor
2027872,2027873,2018-03-17 17:26:01,2018-03-17,17:26:01,44.98,3512837025620287,253,0,False,NaN,3512837025620287,546566,2018-03-09,-15.716701,-47.855818,2017-03-03,Active Life 81
1851620,1851621,2018-03-11 19:42:45,2018-03-11,19:42:45,85.28,30092889305992,254,0,False,NaN,30092889305992,234005,2018-01-16,-15.724543,-47.876550,2017-03-03,Elegant Carpets 94
55485,55486,2018-01-03 10:17:41,2018-01-03,10:17:41,88.96,4867771109093320764,57,0,False,NaN,4867771109093320764,241274,2017-06-30,-15.779615,-47.891527,2016-10-14,Organic Beauty 101
3518696,3518697,2018-05-03 14:47:05,2018-05-03,14:47:05,167.73,5277997722226463,556,0,False,NaN,5277997722226463,676257,2017-05-10,-15.823158,-47.846085,2017-06-07,Elite Hair Salon 126
3228231,3228232,2018-04-24 19:22:43,2018-04-24,19:22:43,42.64,3558870140738788,1818,0,False,NaN,3558870140738788,476320,2017-08-24,-3.047734,-60.020102,2018-03-15,Heritage Walks 74


### VERIFICA SE HA DUPLICATAS

In [215]:
num_duplicates = df.duplicated().sum()
if num_duplicates > 0:
    print(f"Number of duplicates: {num_duplicates}")
    df = df.drop_duplicates(keep='first') 
else:
    print("Number of duplicates: 0")

Number of duplicates: 0


In [216]:
# exitem valores 0, estranho

print(df["tx_amount"].min())
print(df["tx_amount"].max())
print(df["tx_amount"].mean())

0.0
4982.1
75.14510625999998


In [217]:
df['tx_hour'] = df['tx_datetime'].dt.hour
df['tx_day_of_week'] = df['tx_datetime'].dt.dayofweek # Monday=0, Sunday=6
df['tx_day_of_month'] = df['tx_datetime'].dt.day
df['is_weekend'] = df['tx_day_of_week'].isin([5, 6]).astype(int)

print(df[['tx_hour', 'tx_day_of_week', 'is_weekend']].head())

         tx_hour  tx_day_of_week  is_weekend
2027872       17               5           1
1851620       19               6           1
55485         10               2           0
3518696       14               3           0
3228231       19               1           0


In [218]:
import pandas as pd

df_trans = df.sort_values(['card_id', 'tx_datetime', 'transaction_id']).reset_index(drop=True)

df_indexed = df_trans.set_index('tx_datetime')

for time_window in ['1h', '24h', '7d', '30d']:
    col_name = f'card_transactions_last_{time_window}_count'

    temp_series_with_multiindex = df_indexed.groupby('card_id', sort=False)['transaction_id'] \
                                            .rolling(window=time_window) \
                                            .count()


    calculated_values = temp_series_with_multiindex.fillna(0).values


    df[col_name] = calculated_values

print(df["card_transactions_last_7d_count"].head(10))
df

2027872    1.0
1851620    1.0
55485      1.0
3518696    2.0
3228231    1.0
4134869    2.0
3880528    1.0
723699     1.0
3015311    2.0
124544     1.0
Name: card_transactions_last_7d_count, dtype: float64


,transaction_id,tx_datetime,tx_date,tx_time,tx_amount,card_id,terminal_id,is_transactional_fraud,is_fraud,tx_fraud_report_date,...,terminal_operation_start,terminal_soft_descriptor,tx_hour,tx_day_of_week,tx_day_of_month,is_weekend,card_transactions_last_1h_count,card_transactions_last_24h_count,card_transactions_last_7d_count,card_transactions_last_30d_count
2027872,2027873,2018-03-17 17:26:01,2018-03-17,17:26:01,44.98,3512837025620287,253,0,False,NaN,...,2017-03-03,Active Life 81,17,5,17,1,1.0,1.0,1.0,1.0
1851620,1851621,2018-03-11 19:42:45,2018-03-11,19:42:45,85.28,30092889305992,254,0,False,NaN,...,2017-03-03,Elegant Carpets 94,19,6,11,1,1.0,1.0,1.0,2.0
55485,55486,2018-01-03 10:17:41,2018-01-03,10:17:41,88.96,4867771109093320764,57,0,False,NaN,...,2016-10-14,Organic Beauty 101,10,2,3,0,1.0,1.0,1.0,2.0
3518696,3518697,2018-05-03 14:47:05,2018-05-03,14:47:05,167.73,5277997722226463,556,0,False,NaN,...,2017-06-07,Elite Hair Salon 126,14,3,3,0,1.0,1.0,2.0,3.0
3228231,3228232,2018-04-24 19:22:43,2018-04-24,19:22:43,42.64,3558870140738788,1818,0,False,NaN,...,2018-03-15,Heritage Walks 74,19,1,24,0,1.0,1.0,1.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1723002,1723003,2018-03-07 12:38:43,2018-03-07,12:38:43,118.04,3525014257627027,1291,0,False,NaN,...,2018-01-23,Luxury Fashion 9,12,2,7,0,1.0,1.0,2.0,5.0
3756719,3756720,2018-05-10 16:16:23,2018-05-10,16:16:23,168.05,3505230458816508,1328,0,False,NaN,...,2018-01-27,Future Vision 4,16,3,10,0,1.0,1.0,1.0,6.0
3071901,3071902,2018-04-20 06:32:50,2018-04-20,06:32:50,45.33,4842526405179589,1244,0,False,NaN,...,2018-01-15,Digital Paradise 37,6,4,20,0,1.0,1.0,1.0,1.0
2359135,2359136,2018-03-28 14:08:24,2018-03-28,14:08:24,14.87,2274211889594915,635,0,False,NaN,...,2017-06-28,Adventure Park 20,14,2,28,0,1.0,1.0,2.0,2.0


In [219]:
df['card_historical_avg_amount'] = df.groupby('card_id')['tx_amount'] \
                                     .transform(lambda x: x.expanding().mean().shift(1)) \
                                     .fillna(df['tx_amount']) # Fill NaN for first transaction with current amount or 0

df['amount_deviation_card_historical_mean'] = df['tx_amount'] - df['card_historical_avg_amount']

In [220]:
df

,transaction_id,tx_datetime,tx_date,tx_time,tx_amount,card_id,terminal_id,is_transactional_fraud,is_fraud,tx_fraud_report_date,...,tx_hour,tx_day_of_week,tx_day_of_month,is_weekend,card_transactions_last_1h_count,card_transactions_last_24h_count,card_transactions_last_7d_count,card_transactions_last_30d_count,card_historical_avg_amount,amount_deviation_card_historical_mean
2027872,2027873,2018-03-17 17:26:01,2018-03-17,17:26:01,44.98,3512837025620287,253,0,False,NaN,...,17,5,17,1,1.0,1.0,1.0,1.0,44.980000,0.000000
1851620,1851621,2018-03-11 19:42:45,2018-03-11,19:42:45,85.28,30092889305992,254,0,False,NaN,...,19,6,11,1,1.0,1.0,1.0,2.0,85.280000,0.000000
55485,55486,2018-01-03 10:17:41,2018-01-03,10:17:41,88.96,4867771109093320764,57,0,False,NaN,...,10,2,3,0,1.0,1.0,1.0,2.0,88.960000,0.000000
3518696,3518697,2018-05-03 14:47:05,2018-05-03,14:47:05,167.73,5277997722226463,556,0,False,NaN,...,14,3,3,0,1.0,1.0,2.0,3.0,167.730000,0.000000
3228231,3228232,2018-04-24 19:22:43,2018-04-24,19:22:43,42.64,3558870140738788,1818,0,False,NaN,...,19,1,24,0,1.0,1.0,1.0,3.0,42.640000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1723002,1723003,2018-03-07 12:38:43,2018-03-07,12:38:43,118.04,3525014257627027,1291,0,False,NaN,...,12,2,7,0,1.0,1.0,2.0,5.0,94.459286,23.580714
3756719,3756720,2018-05-10 16:16:23,2018-05-10,16:16:23,168.05,3505230458816508,1328,0,False,NaN,...,16,3,10,0,1.0,1.0,1.0,6.0,45.627436,122.422564
3071901,3071902,2018-04-20 06:32:50,2018-04-20,06:32:50,45.33,4842526405179589,1244,0,False,NaN,...,6,4,20,0,1.0,1.0,1.0,1.0,76.645091,-31.315091
2359135,2359136,2018-03-28 14:08:24,2018-03-28,14:08:24,14.87,2274211889594915,635,0,False,NaN,...,14,2,28,0,1.0,1.0,2.0,2.0,12.937576,1.932424


In [221]:
df_indexed = df_trans.set_index('tx_datetime')
for time in ["1h", "24h", "7d", "30d"]:
    amounts = df_indexed.groupby('card_id')['tx_amount'] \
                        .transform(lambda x: x.expanding().mean().shift(1)) \
                        .fillna(df['tx_amount'])
    
    calculated_values = amounts.fillna(0).values
    df[f'card_avg_amount_last_{time}'] = calculated_values
    df[f'card_diff_amount_last_{time}'] = df['tx_amount'] - df[f'card_avg_amount_last_{time}']

In [222]:
df

,transaction_id,tx_datetime,tx_date,tx_time,tx_amount,card_id,terminal_id,is_transactional_fraud,is_fraud,tx_fraud_report_date,...,card_historical_avg_amount,amount_deviation_card_historical_mean,card_avg_amount_last_1h,card_diff_amount_last_1h,card_avg_amount_last_24h,card_diff_amount_last_24h,card_avg_amount_last_7d,card_diff_amount_last_7d,card_avg_amount_last_30d,card_diff_amount_last_30d
2027872,2027873,2018-03-17 17:26:01,2018-03-17,17:26:01,44.98,3512837025620287,253,0,False,NaN,...,44.980000,0.000000,0.000000,44.980000,0.000000,44.980000,0.000000,44.980000,0.000000,44.980000
1851620,1851621,2018-03-11 19:42:45,2018-03-11,19:42:45,85.28,30092889305992,254,0,False,NaN,...,85.280000,0.000000,42.140000,43.140000,42.140000,43.140000,42.140000,43.140000,42.140000,43.140000
55485,55486,2018-01-03 10:17:41,2018-01-03,10:17:41,88.96,4867771109093320764,57,0,False,NaN,...,88.960000,0.000000,52.840000,36.120000,52.840000,36.120000,52.840000,36.120000,52.840000,36.120000
3518696,3518697,2018-05-03 14:47:05,2018-05-03,14:47:05,167.73,5277997722226463,556,0,False,NaN,...,167.730000,0.000000,42.193333,125.536667,42.193333,125.536667,42.193333,125.536667,42.193333,125.536667
3228231,3228232,2018-04-24 19:22:43,2018-04-24,19:22:43,42.64,3558870140738788,1818,0,False,NaN,...,42.640000,0.000000,51.847500,-9.207500,51.847500,-9.207500,51.847500,-9.207500,51.847500,-9.207500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1723002,1723003,2018-03-07 12:38:43,2018-03-07,12:38:43,118.04,3525014257627027,1291,0,False,NaN,...,94.459286,23.580714,64.791538,53.248462,64.791538,53.248462,64.791538,53.248462,64.791538,53.248462
3756719,3756720,2018-05-10 16:16:23,2018-05-10,16:16:23,168.05,3505230458816508,1328,0,False,NaN,...,45.627436,122.422564,62.787143,105.262857,62.787143,105.262857,62.787143,105.262857,62.787143,105.262857
3071901,3071902,2018-04-20 06:32:50,2018-04-20,06:32:50,45.33,4842526405179589,1244,0,False,NaN,...,76.645091,-31.315091,66.117333,-20.787333,66.117333,-20.787333,66.117333,-20.787333,66.117333,-20.787333
2359135,2359136,2018-03-28 14:08:24,2018-03-28,14:08:24,14.87,2274211889594915,635,0,False,NaN,...,12.937576,1.932424,65.360625,-50.490625,65.360625,-50.490625,65.360625,-50.490625,65.360625,-50.490625
